In [2]:
import numpy as np
from fractions import Fraction

def _strength_to_infidelity(basis_fidelity, approximate=False):
    """
    Converts a dictionary mapping XX strengths to fidelities to a dictionary mapping XX
    strengths to infidelities. Also supports one of the other formats Qiskit uses: if only a
    lone float is supplied, it extends it from CX over CX/2 and CX/3 by linear decay.
    """

    if isinstance(basis_fidelity, float):
        if not approximate:
            slope, offset = 1e-10, 1e-12
        else:
            slope, offset = (1 - basis_fidelity) / 2, (1 - basis_fidelity) / 2
        return {
            strength: slope * strength / (np.pi / 2) + offset
            for strength in [np.pi / 2, np.pi / 4, np.pi / 6]
        }
    elif isinstance(basis_fidelity, dict):
        return {
            strength: (1 - fidelity if approximate else 1e-12 + 1e-10 * strength / (np.pi / 2))
            for (strength, fidelity) in basis_fidelity.items()
        }

    raise TypeError("Unknown basis_fidelity payload.")

def default_zx_operation_cost(
    strength: Fraction,
    # note: Isaac reports this value in percent per degree
    scale_factor: float = (64 * 90) / (10000 * 100),
    # first component: 2Q invocation cost; second component: local cost
    offset: float = 909 / (10000 * 100) + 1 / 1000,
):
    """A sample fidelity cost model, extracted from experiment, for ZX
    operations."""
    return strength * scale_factor + offset

In [ ]:
0.

In [23]:
default_zx_operation_cost(strength=Fraction(1)), default_zx_operation_cost(strength=Fraction(1, 2)), default_zx_operation_cost(strength=Fraction(1, 3))  # Example usage

(0.0076690000000000005, 0.004789, 0.003829)

In [29]:
_strength_to_infidelity(0.995, approximate=True)

{1.5707963267948966: 0.0050000000000000044,
 0.7853981633974483: 0.0037500000000000033,
 0.5235987755982988: 0.003333333333333336}